## SFT Training

Here we apply the prepared data to the model and train.

### 1. Configuration

In [4]:
from transformers import BitsAndBytesConfig
# import flash_attention
import torch

model_name = "/ssdshare/Phi-3-mini-128k-instruct"
device_map = "auto"

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

# Load base model
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    # attn_implementation="flash_attention_2",  # loading the model with flash-attention support
    torch_dtype=torch.bfloat16,
    device_map=device_map,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    **model_kwargs,
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 2048
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = "right" # Phi 3 uses right padding

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
# Run text generation pipeline with loaded model
prompt = "Hello Furina! What are your plans for today?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

The model 'Phi3ForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCaus

<s>[INST] Hello Furina! What are your plans for today? [/INST] As an artificial intelligence, I don't have personal plans or feelings. However, I am programmed to assist users with their inquiries and provide information or perform tasks as requested.


### 2. Setting arguments for training

In [2]:
# The base model
model_name = "/ssdshare/Phi-3-mini-128k-instruct"
# The instruction dataset to use
dataset_name = "../data/furina_py18k"
# Fine-tuned model name
new_model = "Phi-3-furina"
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64
# Alpha parameter for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA layers
lora_dropout = 0.1
bias="none"
task_type="CAUSAL_LM"

################################################################################
# Training parameters (passed to TrainingArguments)
################################################################################

# Number of training epochs
num_train_epochs = 1
# Number of training steps (overrides num_train_epochs)
# max_steps = 100
# Enable fp16/bf16 training (set bf16 to True if supported by your GPU)
fp16 = False
bf16 = True
# Batch size per GPU for training
per_device_train_batch_size = 4
# Batch size per GPU for evaluation
per_device_eval_batch_size = 4
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1
# Enable gradient checkpointing
gradient_checkpointing = True
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule
lr_scheduler_type = "cosine"
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 0

################################################################################
# Monitoring parameters
################################################################################

# Logging dir (for tensorboard)
logging_dir = f"{output_dir}/logs"
# Log every X updates steps
logging_steps = 25
# Monitoring and Visualizing tools
report_to = "tensorboard"

################################################################################
# SFTTrainer parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 512
# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

### 3. Start training

In [4]:
# Load dataset
from datasets import load_from_disk
dataset = load_from_disk(dataset_name)
print(dataset[0])

{'text': "<s>[INST] What would you say in the scenario given? The given input is About the Vision.. [/INST] ```Once the ancient prophecy came to an end and everything was over, I fell into low spirits for a very long time. People who stand on the stage basking in the adulation of their audience must also bear the pressures of being in the public eye and living up to everyone's expectations. But I knew very well my person wouldn't be enough, and that putting on the act of a god would be able the only way to satisfy their adoration... And at the end of the day, all I ever experienced was loneliness. So there came a point when I loathed the very thought of acting and locked myself in my room. It wasn't until the moment I stood on stage and faced the audience again that I realized the anxiety in my heart had dissipated. The reason I can now stand before the crowd's watchful gaze is perhaps because... I have finally started to act as myself.``` </s>"}


In [6]:
from peft import PeftModel
from trl import SFTTrainer

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    #max_steps=max_steps,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    logging_steps=logging_steps,
    logging_dir=logging_dir,
    report_to=report_to, 
)

In [6]:
from peft import LoraConfig
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias=bias,
    task_type=task_type,
    target_modules="all linear",
)

In [7]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

In [8]:
# Train the model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Step,Training Loss


In [14]:
# release GPU memory here
torch.cuda.reset_peak_memory_stats()
torch.cuda.empty_cache()

### 3. Merge new model with LoRA

In [11]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=device_map,
    trust_remote_code=True,
)
model = PeftModel.from_pretrained(base_model, new_model)
merged_model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

# Save merged model to disk (optional)
merged_model.save_pretrained(f'{new_model}_merged')

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Phi-3-furina/resolve/main/adapter_model.safetensors (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f9f1a2fc850>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: ced89d20-101f-47f9-9abe-a8d5ba119f52)')

In [12]:
# Load new model
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    # attn_implementation="flash_attention_2",  # loading the model with flash-attention support
    torch_dtype=torch.bfloat16,
    device_map=device_map,
)
model = AutoModelForCausalLM.from_pretrained(
    new_model,
    **model_kwargs,
)
prompt = "Hello Furina! What are your plans for today?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Phi-3-furina/resolve/main/adapter_model.safetensors (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f9f17dfc7c0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: 0912fb6c-2063-4dd0-b197-511d99a1f3b3)')